In [76]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

In [77]:
# import data from stages collection (includes annotated and not annotated)
df = pd.read_csv('/home/anau/SOD_labeling/experiment_2/backups/stages.csv.20221114')
display(df.head())
print(df.shape)

,image,label,bodypart
0,/anau_img3/fc0/fc010202.13.icon.JPG,1,head
1,/anau_img3/fc0/fc010202.14.icon.JPG,1,head
2,/anau_img3/fc0/fc010202.15.icon.JPG,1,head
3,/anau_img3/fc0/fc010202.42.icon.JPG,1,head
4,/anau_img3/fc0/fc010202.43.icon.JPG,1,head


(9999, 3)


In [85]:
# get only the annotated
classes = ['1','2','3','4','5','6','7','8','9','10','11','12','13']
df_anot = df[(df.label.isin(classes)) & (df.bodypart == 'head')]

In [86]:
print(df_anot.shape)
display(df_anot.head())

(5079, 3)


,image,label,bodypart
0,/anau_img3/fc0/fc010202.13.icon.JPG,1,head
1,/anau_img3/fc0/fc010202.14.icon.JPG,1,head
2,/anau_img3/fc0/fc010202.15.icon.JPG,1,head
3,/anau_img3/fc0/fc010202.42.icon.JPG,1,head
4,/anau_img3/fc0/fc010202.43.icon.JPG,1,head


In [87]:
df_anot['image_short'] = df_anot['image'].str.split('.').str[0].copy()
display(df_anot.head())
print(df_anot.shape)

<ipython-input-87-ae2c367c6693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anot['image_short'] = df_anot['image'].str.split('.').str[0].copy()


,image,label,bodypart,image_short
0,/anau_img3/fc0/fc010202.13.icon.JPG,1,head,/anau_img3/fc0/fc010202
1,/anau_img3/fc0/fc010202.14.icon.JPG,1,head,/anau_img3/fc0/fc010202
2,/anau_img3/fc0/fc010202.15.icon.JPG,1,head,/anau_img3/fc0/fc010202
3,/anau_img3/fc0/fc010202.42.icon.JPG,1,head,/anau_img3/fc0/fc010202
4,/anau_img3/fc0/fc010202.43.icon.JPG,1,head,/anau_img3/fc0/fc010202


(5079, 4)


In [88]:
# count number of unique labels per image_short which should be one if annotation was done correctly
# if > 1 then images of same donor taken on same day were labeled differently which should not be the case
df_anot_grouped = df_anot.groupby(['image_short'])['label'].nunique().reset_index(name='count')
display(df_anot_grouped.head())
print(df_anot_grouped.shape)

# get image_short where count > 1 -> these are wrongly annotated
df_to_fix = df_anot_grouped[df_anot_grouped['count'] > 1]
display(df_to_fix.head())
print(df_to_fix.shape)

,image_short,count
0,/anau_img3/000/00000122,1
1,/anau_img3/000/00000129,1
2,/anau_img3/000/00000213,1
3,/anau_img3/000/00000219,1
4,/anau_img3/000/00000222,1


(4085, 2)


,image_short,count
12,/anau_img3/006/00600112,2
35,/anau_img3/00e/00e10226,2
43,/anau_img3/010/01000523,2
78,/anau_img3/014/01410214,2
117,/anau_img3/020/02000904,2


(134, 2)


In [83]:
df_anot[df_anot.image_short == '/anau_img3/00d/00d01206']

,image,label,bodypart,image_short
2217,/anau_img3/00d/00d01206.08.icon.JPG,2,head,/anau_img3/00d/00d01206
2496,/anau_img3/00d/00d01206.07.icon.JPG,2,head,/anau_img3/00d/00d01206


In [84]:
# join df_to_fix with df_anot to get full image paths
df_to_fix2 = df_to_fix.merge(df_anot, left_on='image_short', right_on='image_short', how='left')
df_to_fix2
display(df_to_fix2.head())
print(df_to_fix2.shape)

,image_short,bodypart_x,count,image,label,bodypart_y
0,/anau_img3/006/00600112,head,2,/anau_img3/006/00600112.12.icon.JPG,3,head
1,/anau_img3/006/00600112,head,2,/anau_img3/006/00600112.11.icon.JPG,2,head
2,/anau_img3/00e/00e10226,head,2,/anau_img3/00e/00e10226.21.icon.JPG,7,head
3,/anau_img3/00e/00e10226,head,2,/anau_img3/00e/00e10226.08.icon.JPG,3,head
4,/anau_img3/010/01000523,head,2,/anau_img3/010/01000523.33.icon.JPG,6,head


(348, 6)


In [74]:
# 
df_to_fix3 = df_to_fix2.groupby(['image_short','count']).agg({'image':lambda x: list(x), 
                                                              'label': lambda x: list(x)}).reset_index()
display(df_to_fix3)
print(df_to_fix3.shape)

,image_short,count,image,label
0,/anau_img3/006/00600112,2,"[/anau_img3/006/00600112.12.icon.JPG, /anau_img3/006/00600112.11.icon.JPG]","[3, 2]"
1,/anau_img3/00e/00e10226,2,"[/anau_img3/00e/00e10226.21.icon.JPG, /anau_img3/00e/00e10226.08.icon.JPG]","[7, 3]"
2,/anau_img3/010/01000523,2,"[/anau_img3/010/01000523.33.icon.JPG, /anau_img3/010/01000523.31.icon.JPG]","[6, 5]"
3,/anau_img3/014/01410214,2,"[/anau_img3/014/01410214.08.icon.JPG, /anau_img3/014/01410214.09.icon.JPG]","[6, 8]"
4,/anau_img3/020/02000904,2,"[/anau_img3/020/02000904.16.icon.JPG, /anau_img3/020/02000904.40.icon.JPG]","[11, 10]"
5,/anau_img3/038/03800906,2,"[/anau_img3/038/03800906.51.icon.JPG, /anau_img3/038/03800906.50.icon.JPG]","[8, 7]"
6,/anau_img3/040/04000723,2,"[/anau_img3/040/04000723.06.icon.JPG, /anau_img3/040/04000723.05.icon.JPG]","[9, 7]"
7,/anau_img3/06a/06a01210,2,"[/anau_img3/06a/06a01210.08.icon.JPG, /anau_img3/06a/06a01210.08.icon.JPG]","[9, 5]"
8,/anau_img3/07b/07b00325,2,"[/anau_img3/07b/07b00325.31.icon.JPG, /anau_img3/07b/07b00325.29.icon.JPG]","[5, 4]"
9,/anau_img3/082/08200315,2,"[/anau_img3/082/08200315.38.icon.JPG, /anau_img3/082/08200315.33.icon.JPG]","[7, 9]"


(134, 4)


In [64]:
df_to_fix3.to_csv('stages.csv.20221114_to_fix', header=False, index=False)